# SPRINT - 3

## Item - ?

<br>
<br>

Autor.: Sérgio C. Medina

#### Declaração dos Pacotes, Libs ou Classes utilizadas no processo.

In [1]:
# Declaração dos Pacotes, Libs ou Classes utilizadas no processo.
import os
import io
import math
import pandas as pd
import gcsfs
import pyarrow
import pyarrow.parquet as pq
from google.cloud import storage
from datetime import datetime, timedelta

import sys
sys.path.append('../../pods')

# configurando variavel de ambiente com o arquivo de credenciais para conexão GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "..\..\secrets\edc-igti-smedina-4920e12ac565.json"
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/sergiomedina/Downloads/edc-igti-smedina-4920e12ac565.json"

# Funções de integração com o Cloud Storage
from utilGCS import utilGCS
gcs = utilGCS(bucketName='edc-pa-i4-data')

# Funções diversas de manipulação de dados
from utilFuncs import utilFuncs
func = utilFuncs()


#### Dados na "processing-zone" pasta "DATAOP"

In [18]:
dtexec = "2021-11-09"
source = "dataop"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataop = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataop = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])




In [20]:
df_dataop.dtypes

OP                 object
DTINI      datetime64[ns]
DTFIM      datetime64[ns]
CODMAT             object
LOTEFAB            object
QTDPLAN             int64
DTPROD           category
dtype: object

#### Dados na "raw-data-zone" pasta "DATACONFIRM"

In [ ]:
dtexec = "2021-11-08"
source = "dataconfirm"
folder = f"raw-data-zone/{source}"

#df_dataconfirm = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataconfirm = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])


In [ ]:
df_dataconfirm

In [ ]:
df_dataconfirm.groupby(
    ['DTPROD', 'IDTURNO', 'LOTE']
).agg(
    {
        'PACKID': 'count',
        'UNIDADES': 'sum',
        'KGPACK': 'sum',
        'KGUNMED': 'mean'
    }
)


# df = pd.DataFrame(df_dataconfirm.groupby(
#     ['DTPROD', 'IDTURNO', 'LOTE']
# ).agg(
#     {
#         'PACKID': 'count',
#         'UNIDADES': 'sum',
#         'KGPACK': 'sum',
#         'KGUNMED': 'mean'
#     }
# )).reset_index()

# df

#### Dados na "raw-data-zone" pasta "DATAPROD"

In [ ]:
dtexec = "2021-11-09"
lineid = "101"
source = "dataprod"
folder = f"raw-data-zone/{source}"

#df_dataprod = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataprod = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])


In [ ]:
df_dataprod.dtypes

In [ ]:
#df.groupby(['col1', 'col2']).agg({'col3':'sum','col4':'sum'})
df_sts = pd.DataFrame(df_dataprod.groupby(
    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH', 'STSID', 'STSDS']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum'
    }
)).reset_index()


df = df_dataprod.copy()

df['TMIN'] = df['TIMESTAMP']
df['TMAX'] = df['TIMESTAMP']
df.drop(['TIMESTAMP'], axis=1, inplace=True)

df = pd.DataFrame(df.groupby(
    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum',
        'TMIN': 'min',
        'TMAX': 'max'
    }
)).reset_index()


df['TMAXLIMIT'] = df.apply(lambda row:func.end_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMAXLIMIT'] = pd.to_datetime(df['TMAXLIMIT'])
df['TMAXDIFF'] = (df['TMAXLIMIT']-df['TMAX']).dt.seconds/60
df['WIP'] = df.apply(lambda row:func.round_down(row['TMAXDIFF'], 0),axis=1)

df['TMINLIMIT'] = df.apply(lambda row:func.start_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMINLIMIT'] = pd.to_datetime(df['TMINLIMIT'])
df['TMINDIFF'] = (df['TMIN']-df['TMINLIMIT']).dt.seconds/60

df['DIFF'] = df['TMAXDIFF'].diff().fillna(0)

df['TOTMINADJUSTED'] = round(df['TOTMIN'] + df['DIFF'],0)

df.drop(['TMAX', 'TMAXLIMIT', 'TMAXDIFF'], axis=1, inplace=True)
df.drop(['TMIN', 'TMINLIMIT', 'TMINDIFF', 'DIFF'], axis=1, inplace=True)

# Tot. Tempo turno
df['TPTOTAL'] = 480

df['TOTMINADJUSTED'] = df.apply(lambda row:(row['TOTMINADJUSTED'] if row['TOTMINADJUSTED']<480 else 480),axis=1)

# Tot. Tempo = WORKING
df['TPWORKING'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==4) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

df['TPWORKING'] = df.apply(lambda row:(row['TPWORKING'] if row['TPWORKING']<=row['TOTMINADJUSTED'] else row['TOTMINADJUSTED']),axis=1)

# Tot. Tempo = TPSTOPPLAN
df['TPSTOPPLAN'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

# Tot. Tempo = TPNOALLOC
df['TPNOALLOC'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==0) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)    


# Tempo Programado para produzir
df['TPPROG'] = df['TPTOTAL'] - (df['TPNOALLOC']+df['TPSTOPPLAN'])


# Temp Ociosidade
df['TPIDLE'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID>=1) &
    (df_sts.STSID<3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

df['TPIDLE'] = ((df['TPPROG']-(df['TPWORKING'] + df['TPIDLE']))+df['TPIDLE'])

# Unidades - Produção Teórica
df['PCTHEOR'] = df['TPPROG'] * 1

# Unidades - Perda por performance
df['PCLOSS'] = df['PCTHEOR'] - df['PC']


# OEE Calc
# df['OEEDISP'] = df['TPWORKING'] / df['TPPROG']

# df['OEEPERF'] = df['PC'] / df['PCTHEOR']

# df['OEEQUAL'] = df['GOOD'] / df['PC']

# df['OEE'] = df['OEEDISP'] * df['OEEPERF'] * df['OEEQUAL']




In [ ]:
df

In [ ]:
df.agg(
    {
        'OEEDISP': 'prod',
        'OEEPERF': 'prod',
        'OEEQUAL': 'prod',
        'OEE': 'prod'
    }
)